# VacationPy
---

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/my_cities.csv")

# Display sample data
city_data_df.head()

,City_ID,city,latitude,longitude,max temp,city humidity,cloudiness,wind speed,country,date
0,0,adamstown,-25.0660,-130.1015,77.31,82,100,13.76,PN,1710362300
1,1,puerto natales,-51.7236,-72.4875,48.65,61,75,21.85,CL,1710362301
2,2,college,64.8569,-147.8028,4.17,55,40,1.99,US,1710362301
3,3,pali,25.7667,73.3333,68.52,35,0,4.52,IN,1710362301
4,4,kristiansund,63.1115,7.7320,44.65,87,75,12.66,NO,1710362302


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot = city_data_df.hvplot.points('longitude', 'latitude', geo= True, tiles = 'OSM',
                                      frame_width = 700, frame_height = 500, size = 'city humidity', color= 'city')

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (city,city humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit my vacation criteria: Max temperature between 95 and 80 degrees, wind speed less than 4 m/s, and cloudiness <10.
vacation_filter_df = city_data_df.loc[(city_data_df['max temp']>=75) & (city_data_df['max temp']<=100) & (city_data_df['wind speed']<5) & (city_data_df['cloudiness'] <15)]
vacation_filter_df

,City_ID,city,latitude,longitude,max temp,city humidity,cloudiness,wind speed,country,date
5,5,santa maria,-29.6842,-53.8069,88.90,66,0,4.61,BR,1710362302
28,28,la'ie,21.6477,-157.9253,82.36,88,2,1.01,US,1710362260
60,60,banda aceh,5.5577,95.3222,78.58,77,12,4.47,ID,1710362317
62,62,ugoofaaru,5.6667,73.0000,81.77,73,10,4.23,MV,1710362318
99,99,salalah,17.0151,54.0924,80.69,78,0,2.30,OM,1710362328
182,182,ad darb,17.7229,42.2526,84.11,72,1,3.85,SA,1710362349
295,295,belo sur tsiribihina,-19.7000,44.5500,78.91,86,6,2.13,MG,1710362381
385,385,puerto eldorado,-26.4084,-54.6946,95.94,34,6,1.03,AR,1710362409


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_filter_df.copy(deep= True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['hotel name'] = ""

# Display to verify new column name
hotel_df

,City_ID,city,latitude,longitude,max temp,city humidity,cloudiness,wind speed,country,date,hotel name
5,5,santa maria,-29.6842,-53.8069,88.90,66,0,4.61,BR,1710362302,
28,28,la'ie,21.6477,-157.9253,82.36,88,2,1.01,US,1710362260,
60,60,banda aceh,5.5577,95.3222,78.58,77,12,4.47,ID,1710362317,
62,62,ugoofaaru,5.6667,73.0000,81.77,73,10,4.23,MV,1710362318,
99,99,salalah,17.0151,54.0924,80.69,78,0,2.30,OM,1710362328,
182,182,ad darb,17.7229,42.2526,84.11,72,1,3.85,SA,1710362349,
295,295,belo sur tsiribihina,-19.7000,44.5500,78.91,86,6,2.13,MG,1710362381,
385,385,puerto eldorado,-26.4084,-54.6946,95.94,34,6,1.03,AR,1710362409,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    city = row["city"]
    latitude = row["latitude"]
    longitude = row["longitude"]
    
    # Set up parameters for the request
    params = {
        "lat": latitude,
        "lon": longitude,
        "categories": categories,
        "apiKey": geoapify_key
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    place_details = response.json()

    # Print the json (pretty printed)
    # print(json.dumps(place_details, indent=4, sort_keys=True))
    
    # Extract the hotel name from the response
   
    try:
        name = place_details["features"][0]["properties"]["name"]
    except KeyError:
        name = "No hotel found"
    print(name)
    # Log the search results
    print(f"{city} - nearest hotel: {hotel_name}")

    # Store the hotel name in the DataFrame
    hotel_df.loc[index, "hotel name"] = name
    # break


Starting hotel search
Hotel Morotin
santa maria - nearest hotel: No hotel found
No hotel found
la'ie - nearest hotel: No hotel found
Hotel 61
banda aceh - nearest hotel: No hotel found
Cora Cora Maldives
ugoofaaru - nearest hotel: No hotel found
Muscat International Hotel
salalah - nearest hotel: No hotel found
أباهي الدرب للوحدات السكنية المفروشة
ad darb - nearest hotel: No hotel found
Karibo
belo sur tsiribihina - nearest hotel: No hotel found
Che Roga
puerto eldorado - nearest hotel: No hotel found


In [15]:
# Display sample data
hotel_df

,City_ID,city,latitude,longitude,max temp,city humidity,cloudiness,wind speed,country,date,hotel name
5,5,santa maria,-29.6842,-53.8069,88.90,66,0,4.61,BR,1710362302,Hotel Morotin
28,28,la'ie,21.6477,-157.9253,82.36,88,2,1.01,US,1710362260,No hotel found
60,60,banda aceh,5.5577,95.3222,78.58,77,12,4.47,ID,1710362317,Hotel 61
62,62,ugoofaaru,5.6667,73.0000,81.77,73,10,4.23,MV,1710362318,Cora Cora Maldives
99,99,salalah,17.0151,54.0924,80.69,78,0,2.30,OM,1710362328,Muscat International Hotel
182,182,ad darb,17.7229,42.2526,84.11,72,1,3.85,SA,1710362349,أباهي الدرب للوحدات السكنية المفروشة
295,295,belo sur tsiribihina,-19.7000,44.5500,78.91,86,6,2.13,MG,1710362381,Karibo
385,385,puerto eldorado,-26.4084,-54.6946,95.94,34,6,1.03,AR,1710362409,Che Roga


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
# Configure the map plot
map_plot2 = hotel_df.hvplot.points('longitude', 'latitude', geo= True, tiles = 'OSM',
                                      frame_width = 700, frame_height = 500, size = 'city humidity', color= 'city', hover_cols=['hotel name', 'country'])

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (city,city humidity,hotel name,country)